In [1]:
import warnings
warnings.filterwarnings('ignore') # :clown:

In [2]:
import pandas as pd
import numpy as np

In [20]:
import tensorflow as tf

## Building the Model

In [4]:
import tensorflow as tf
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Dense, LSTM

In [5]:
sampling_frequency = 1000
window = sampling_frequency * 1

In [6]:
inputs = tf.keras.Input(shape=(1,))

expand = tf.expand_dims(inputs, axis=2)
lstm = tf.compat.v1.keras.layers.LSTM(256, return_sequences=True)(expand)
lstm = tf.keras.layers.Dropout(0.2)(lstm)
lstm = tf.compat.v1.keras.layers.LSTM(256, return_sequences=True)(lstm)
flatten = tf.keras.layers.Flatten()(lstm)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.expand_dims (TFOpLambda  (None, 1, 1)              0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 1, 256)            264192    
                                                                 
 dropout (Dropout)           (None, 1, 256)            0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 256)            525312    
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                           

In [7]:
from sklearn.preprocessing import OrdinalEncoder

In [16]:
df_train = pd.read_hdf('./data/dataset.h5', 'train')
# df_test = pd.read_hdf('./data/dataset.h5', 'test')
df_val = pd.read_hdf('./data/dataset.h5', 'val')

In [9]:
df_train.dtypes

timestamp      object
ECG_Clean     float64
subject_id     object
category       object
dtype: object

In [10]:
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'], format='ISO8601')
df_train['subject_id'] = df_train['subject_id'].astype('int32')
df_train['category'] = df_train['category'].astype('category')
df_train.dtypes
df_train.set_index(['timestamp'], inplace=True)

In [11]:
category_encoder = OrdinalEncoder()
df_train['category'] = category_encoder.fit_transform(df_train[['category']])

In [19]:
tensor.window(10)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'window'

In [17]:
# if keep_short_tails is set to True, the slices shorter than window_size at the end of the result will be kept 
def window_split(x, window_size=100, stride=50, keep_short_tails=True):
  length = x.size(0)
  splits = []

  if keep_short_tails:
    for slice_start in range(0, length, stride):
      slice_end = min(length, slice_start + window_size)
      splits.append(x[slice_start:slice_end])
  else:
    for slice_start in range(0, length - window_size + 1, stride):
      slice_end = slice_start + window_size
      splits.append(x[slice_start:slice_end])

  return splits

TypeError: in user code:

    File "C:\Users\Admin\AppData\Local\Temp\ipykernel_5604\3727395440.py", line 2, in get_window  *
        return tf.convert_to_tensor(df_train['ECG_Clean'][starting_idx : starting_idx + 16])
    File "c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\series.py", line 1065, in __getitem__
        return self._getitem_slice(key)
    File "c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\generic.py", line 4284, in _getitem_slice
        slobj = self.index._convert_slice_indexer(key, kind="getitem")
    File "c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\indexes\base.py", line 4246, in _convert_slice_indexer
        self._validate_indexer("slice", key.start, "getitem")
    File "c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\indexes\base.py", line 6636, in _validate_indexer
        self._raise_invalid_indexer(form, k

In [14]:
tensor.map(10)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'map'

In [56]:
X_train = df_train[['ECG_Clean']]
y_train = df_train[['category']]
X_val = df_val[['ECG_Clean']]
y_val = df_val[['category']]

In [57]:
y_train[y_train['category'].isin(['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat'])] = 'Stressed'
y_train[~y_train['category'].isin(['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat'])] = 'Not Stressed'
y_val[y_val['category'].isin(['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat'])] = 'Stressed'
y_val[~y_val['category'].isin(['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat'])] = 'Not Stressed'

In [58]:
X_train['ECG_Clean'] = tf.cast(X_train['ECG_Clean'], dtype=tf.float32)
X_val['ECG_Clean'] = tf.cast(X_val['ECG_Clean'], dtype=tf.float32)

In [59]:
y_val = category_encoder.transform(y_val[['category']])

ValueError: Found unknown categories ['Not Stressed'] in column 0 during transform

In [62]:
model.fit(X_train, y_train, epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100




UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1046, in launch_instance

  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Users\Admin\AppData\Local\Programs\Pyth

In [18]:
# model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(
#             monitor='val_loss',
#             patience=5,
#             restore_best_weights=True
#         )
#     ]
# )

Epoch 1/100


228500/228500 [==============================] - 1699s 7ms/step - loss: 9.4232e-06 - accuracy: 1.0000 - auc: 0.0000e+00 - val_loss: 3.3576e-12 - val_accuracy: 1.0000 - val_auc: 0.0000e+00
Epoch 2/100
128762/228500 [===============>..............] - ETA: 9:30 - loss: 3.8532e-12 - accuracy: 1.0000 - auc: 0.0000e+00

KeyboardInterrupt: 